In [1]:
import os
import pandas as pd
import numpy as np
import fitsio
import cudf
from numba import cuda
import cupy as cp

In [2]:
def extract_coincidentals(spikes_list, idx):
    
    # Spikes coordinates at given wavelength index
    spikes_w = spikes_list[idx]
    # Associated neighbour coordinates
    nb_pixels = index_8nb[spikes_w[0, :], :]
    # Sublist of spikes data that will excludes the one serving as template
    spikes_sublist = spikes_list[:idx]+spikes_list[idx+1:]
    # Coincidental cross-referencing. 
    mask_w_arr = np.array([np.isin(nb_pixels, index_8nb[spikes[0,:], :]).any(axis=1) for spikes in spikes_sublist])
    select_pixels = mask_w_arr.any(axis=0)
    coords_w = spikes_w[0, select_pixels] 
    w_tables = np.insert(mask_w_arr[:, select_pixels], idx, True, axis=0)
    # Retrieve intensity values for the selected coordinates
    intensities = spikes_w[ 1:, select_pixels]
    arr_w = np.concatenate([coords_w[np.newaxis,...], intensities, w_tables], axis=0)
    arr_w = np.insert(arr_w, 3, idx, axis=0)
    
    return arr_w

def extract_all_coincidentals(spikes_list):
    column_names = ['coords' , 'int1', 'int2', 'wref', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
    group_data = np.concatenate([extract_coincidentals(spikes_list, i) for i in range(7)], axis=1)
    coincidental_spikes_df = pd.DataFrame(group_data.T, columns=column_names)
    return coincidental_spikes_df

In [3]:
data_dir = os.environ['SPIKESDATA']
spikes_db = pd.read_parquet(os.path.join(data_dir, 'spikes_df_2010.parquet'), engine='pyarrow')
spikes_db2 = spikes_db.set_index(['GroupNumber', 'Time'])

### Get the filepaths (typically 7) for a given group

In [4]:
################################################################################################
# Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
################################################################################################
# List of relative 2D coordinates for 8-neighbour connectiviy (9-element list). 1st one is the origin pixel.
coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
# Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
ny, nx = [4096, 4096]
coords_1d = np.arange(nx * ny)
coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
coords2d = np.array([coordy, coordx])
# Create the array of 2D coordinates of 8-neighbours associated with each pixel.
# pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
# Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed. Update
# to per-axis clipping if that ever changes for another instrument.
np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
# Convert to 1D coordinates.
index_8nb = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                     dtype='int32', order='C').T
index_8nb.shape

(16777216, 9)

In [5]:
group_n = 0
fpaths = spikes_db2.loc[group_n]['Path'].values
spikes_list = [fitsio.read(os.path.join(data_dir, f)) for f in fpaths]

nspikes = 0
for spikes in spikes_list:
    nspikes += spikes.shape[1]
print('\ntotal spikes = ', nspikes)


total spikes =  151184


In [8]:
%time df0 = extract_all_coincidentals(spikes_list)

CPU times: user 887 ms, sys: 5.22 ms, total: 892 ms
Wall time: 891 ms


In [36]:
print(len(df0))
df0.head()

16339


,coords,int1,int2,wref,w0,w1,w2,w3,w4,w5,w6
0,18917,122,11,0,1,1,0,0,0,0,0
1,19192,124,7,0,1,0,0,1,0,0,0
2,23013,75,10,0,1,1,0,0,0,0,0
3,23287,157,9,0,1,0,0,1,0,0,0
4,27109,38,9,0,1,1,0,0,0,0,0


### Design method to extract unique coincidental events, lifting any ambiguity (conjugates, redundancies, ...)

In [10]:
def get_dist_mat(coords):
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, np.newaxis]
    coords_yb = coords_y[:, np.newaxis]
    dx_broadc = coords_x - coords_xb
    dy_broadc = coords_y - coords_yb
    dist_matrix = dx_broadc**2 + dy_broadc**2
    return dist_matrix

In [11]:
def get_dist_mat_gpu(coords):
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, cp.newaxis]
    coords_yb = coords_y[:, cp.newaxis]
    dx_broadc = coords_x - coords_xb
    dy_broadc = coords_y - coords_yb
    dist_matrix = dx_broadc**2 + dy_broadc**2
    return dist_matrix

In [12]:
def get_dist_mat2(coords):
    c1, c2 = np.meshgrid(coords, coords)
    c1x = c1 % 4096
    c1y = c1 // 4096
    c2x = c2 % 4096
    c2y = c2 // 4096
    diffc = np.stack((c1x-c2x, c1y-c2y), axis=2)
    dist_matrix = np.sum(diffc**2, axis=2)
    return dist_matrix

In [13]:
def get_rows_list(array, w1_idx, w2_idx):
    np_mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
    df_idx = np.nonzero(np_mask)[0]
    coords = array[np_mask, 0]
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, np.newaxis]
    coords_yb = coords_y[:, np.newaxis]
    dx_broadc = coords_x - coords_xb
    dy_broadc = coords_y - coords_yb
    dist_matrix = np.sqrt(dx_broadc**2 + dy_broadc**2)
    select = dist_matrix < 2 
    select2 = np.triu(select, k=1)
    r,c = np.nonzero(select2)
    idx1, idx2 = df_idx[r], df_idx[c]
    return idx1, idx2

In [14]:
def get_rows_list_gpu(array, w1_idx, w2_idx):
    mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
    df_idx = cp.nonzero(mask)[0]
    coords = array[df_idx, 0]
    dist_matrix = get_dist_mat_gpu(coords)
    select = dist_matrix < 2 
    select2 = cp.triu(select, k=1)
    r,c = cp.nonzero(select2)
    idx1, idx2 = df_idx[r], df_idx[c]
    return idx1, idx2

In [15]:
def get_2coincidentals(array, w1_idx, w2_idx):
    idx1, idx2 = get_rows_list(array, w1_idx, w2_idx)
    records = [df0.loc[[i1, i2]][['coords', 'int1', 'int2', 'wref']] for i1, i2 in zip(idx1, idx2)]
    df_records = pd.concat(records, keys=list(range(len(records))))
    return df_records

In [16]:
def get_2coincidentals2(array, w1_idx, w2_idx):
    idx1, idx2 = get_rows_list(array, w1_idx, w2_idx)
    # To remove conjugates
#     keep_mask = arr0[idx1, 3] != arr0[idx2, 3] 
#     recordsf = np.concatenate((arr0[idx1[keep_mask], 0:4], arr0[idx2[keep_mask], 0:4]), axis=1)
    
    records = np.concatenate((array[idx1, 0:4], array[idx2, 0:4]), axis=1)
    df_records = pd.DataFrame(records, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])

    return df_records

In [17]:
def get_2coincidentals_gpu(gpu_array, w1_idx, w2_idx):
    gidx1, gidx2 = get_rows_list_gpu(gpu_array, w1_idx, w2_idx)
    grecords = cp.concatenate([gpu_array[gidx1, 0:4], gpu_array[gidx2, 0:4]], axis=1)
    #grecords2 = cp.asfortranarray(grecords)
    #cudf_records = cudf.DataFrame.from_gpu_matrix(grecords2, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])
    return records

In [40]:
w1_idx = 4
w2_idx = 5
arr0 = df0.values

In [41]:
%time idx1, idx2 = get_rows_list(arr0, w1_idx, w2_idx)

CPU times: user 9.34 ms, sys: 349 µs, total: 9.69 ms
Wall time: 8.42 ms


In [69]:
%time df_records = get_2coincidentals2(arr0, w1_idx, w2_idx)

CPU times: user 9.51 ms, sys: 133 µs, total: 9.64 ms
Wall time: 8.42 ms


In [24]:
print(len(df_records))
df_records.head()

476


,coords1,int1_before,int1_after,wref1,coords2,int2_before,int2_after,wref2
0,18917,122,11,0,23013,75,10,0
1,23013,75,10,0,27109,38,9,0
2,23013,75,10,0,27108,11,0,1
3,27109,38,9,0,27108,11,0,1
4,272628,240,8,0,272629,32,10,0


In [63]:
np_mask = (arr0[:, w1_idx] == 1) & (arr0[:, w2_idx] == 1)
df_idx = np.nonzero(np_mask)[0]
coords = arr0[np_mask, 0]
len(coords)

485

In [65]:
%timeit dist_matrix = get_dist_mat(coords)

450 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Testing GPU versions

In [25]:
gdf0 = cudf.DataFrame.from_pandas(df0)

In [48]:
cuarr0 = cp.asarray(arr0)
print(cuarr0.shape)

(16339, 11)


In [27]:
%time idx1, idx2 = get_rows_list(arr0, w1_idx, w2_idx)

CPU times: user 7.95 ms, sys: 3.99 ms, total: 11.9 ms
Wall time: 10.7 ms


In [49]:
%timeit gidx1, gidx2 = get_rows_list_gpu(cuarr0, w1_idx, w2_idx)

584 µs ± 4.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
array = cuarr0

In [67]:
%%timeit
mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
df_idx = cp.nonzero(mask)[0]

131 µs ± 773 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [51]:
len(df_idx)

485

In [54]:
coords = array[df_idx, 0]

In [61]:
%timeit dist_matrix = get_dist_mat_gpu(coords)

218 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
select = dist_matrix < 2 
select2 = cp.triu(select, k=1)
r,c = cp.nonzero(select2)
idx1, idx2 = df_idx[r], df_idx[c]

In [187]:
gidx1, gidx2 = get_rows_list_gpu(cuarr0, w1_idx, w2_idx)
grecords = cp.concatenate([cuarr0[gidx1, 0:4], cuarr0[gidx2, 0:4]], axis=1)
grecords2 = cp.asfortranarray(grecords)
cudf_records = cudf.DataFrame.from_gpu_matrix(grecords2, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])
cudf_records.head()

,coords1,int1_before,int1_after,wref1,coords2,int2_before,int2_after,wref2
0,18917,122,11,0,23013,75,10,0
1,23013,75,10,0,27109,38,9,0
2,23013,75,10,0,27108,11,0,1
3,27109,38,9,0,27108,11,0,1
4,272628,240,8,0,272629,32,10,0


In [186]:
%%timeit
gidx1, gidx2 = get_rows_list_gpu(cuarr0, w1_idx, w2_idx)
grecords = cp.concatenate([cuarr0[gidx1, 0:4], cuarr0[gidx2, 0:4]], axis=1)
# grecords2 = cp.asfortranarray(grecords)
# cudf_records = cudf.DataFrame.from_gpu_matrix(grecords2, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])
#cudf_records.head()

693 µs ± 43.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%timeit cudf_records = cudf.DataFrame.from_gpu_matrix(grecords, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])

804 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [106]:
array = cuarr0

In [129]:
%%timeit
mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
df_idx = cp.nonzero(mask)[0]
coords = array[df_idx, 0]
coords_x = coords % 4096
coords_y = coords // 4096
coords_xb = coords_x[:, cp.newaxis]
coords_yb = coords_y[:, cp.newaxis]
dx_broadc = coords_x - coords_xb
dy_broadc = coords_y - coords_yb
dist_matrix = cp.sqrt(dx_broadc**2 + dy_broadc**2)

In [118]:
%%timeit
mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
df_idx = cp.nonzero(mask)[0]
coords = array[df_idx, 0]
coords_x = coords % 4096
coords_y = coords // 4096
coords2d = cp.stack([coords_x, coords_y], axis=1)
diff = coords2d[:, cp.newaxis, :] - coords2d
distance_matrix = cp.sqrt(cp.sum(diff**2, axis=2))

404 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [190]:
print(coords_x.shape, coords_xb.shape)

(485,) (485, 1)


In [194]:
%timeit a = coords_x[:, cp.newaxis]

216 ns ± 5.73 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [193]:
%timeit a = coords_x.reshape([*coords.shape, 1])

364 ns ± 6.09 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [101]:
%timeit mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)

48 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [102]:
%timeit df_idx = cp.nonzero(mask)[0]

67.8 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [103]:
%timeit coords = array[df_idx, 0]

41.6 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [104]:
%timeit coords2d = cp.stack(cp.unravel_index(coords, [4096, 4096]), axis=1)

208 µs ± 8.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
%timeit coords2d = cp.stack([coords_x, coords_y], axis=1)

28.5 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [120]:
%timeit a = dx_broadc**2

97.7 µs ± 439 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<function cupy.manipulation.dims.expand_dims(a, axis)>